Reference: https://towardsdatascience.com/web-scraping-indeed-com-e5591790736d

Reference:
https://github.com/rowandl/portfolio/blob/master/Webscraping%20Indeed.com/Webscraping%20Indeed.ipynb

In [ ]:
#!pip install pandas
#!pip install bs4
#!pip install matplotlib
#!pip install lxml

In [16]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import csv
import time

from IPython.display import HTML

In [ ]:
# top 10 canada cities
cities = ['Toronto', 'Montreal', 'Vancouver', 'Calgary', 'Edmonton', 'Ottawa', 'Winnipeg', 'Quebec+City', 'Hamilton', 'Kitchener']
#cities = ['Toronto']

# Set this to a high-value to generate more results. High value might get block by website
max_results_per_city = 20
#max_results_per_city = 2

total_count = len(cities) * max_results_per_city
city_count = 0
page_count = 0

#making a dataframe which collects all the information from my webscraping
#https://www.workopolis.com/jobsearch/find-jobs?ak=data+scientist&l=Toronto&pn=1
url_template = "https://www.workopolis.com/jobsearch/find-jobs?ak=data+scientist&l={}&pn={}"
base_url = "https://www.workopolis.com"

df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary', 'job_description','job_url'])

for city in cities:
    for start in range(1, max_results_per_city, 1):
        url = url_template.format(city,start)
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, "html.parser")
        for b in soup.find_all('article', attrs = {'class': 'JobCard'}):
            try:
                location = b.find('span', attrs = {'class': 'JobCard-location'}).text
            except:
                location = 'NA'

            job_title = b.find('h2', attrs = {'class':'JobCard-title'}).get('title')
            #job_title = job_title.replace("\n","")
            
            job_url = b.find('a', attrs = {'class':'JobCard-titleLink'}).get('href')

            try:
                company = b.find('div', attrs = {'class':'JobCard-company'}).text
                company = company.replace("\n","")
            except:
                company = 'NA'
            try:
                salary = b.find('span', attrs = {'class' : 'Salary'}).text  
            except:
                salary = 'NA'
            
            time.sleep(1) # to prevent web blocking

            url_job = base_url + job_url
            html_job = urllib.request.urlopen(url_job).read()
            soup_job = BeautifulSoup(html_job, "html.parser")

            try:
                description = soup_job.find('div', attrs = {'class': 'ViewJob-description'}).text
            except:
                description = 'NA'

            done = int((city_count * max_results_per_city + page_count)/total_count*100)

            print("%d%%: %s: %s > %s > %s" % (done, city, location, company, job_title))

            if (description != 'NA'):
                df = df.append({"location":location,"company":company, "job_title": job_title, "salary": salary, "job_description": description, "job_url": url_job}, ignore_index=True)
            
            #time.sleep(1) # to prevent web blocking

        page_count += 1

    city_count += 1
    page_count = 0
   
df.drop_duplicates(inplace=True)
df.company.replace(inplace=True,regex=True,to_replace="\n",value="")
df.job_title.replace(inplace=True,regex=True,to_replace="\n",value="")
df.head()

0%: Toronto:  — Toronto, ON > TD Bank > Sr Machine Learning Product Engineer (Layer 6)
0%: Toronto:  — Oakville, ON > Prodigy Game > Applied Scientist, Revenue & Modelling
0%: Toronto:  — Toronto, ON > Spin Master Ltd > Senior Manager, Data Science
0%: Toronto:  — Toronto, ON > TELUS > Data Scientist
0%: Toronto:  — Toronto, ON > Achievers > Data Scientist
0%: Toronto:  — Toronto, ON > QuadReal > Intern, Data Scientist
0%: Toronto:  — Toronto, ON > Cerebri AI > Data Scientist
0%: Toronto:  — Toronto, ON > Hopper > Data Scientist - Lodging Team
0%: Toronto:  — Toronto, ON > Crescendo Technology > Machine Learning Engineer/Data Scientist
0%: Toronto:  — Toronto, ON > TD Bank > Data Scientist II
0%: Toronto:  — Oakville, ON > Prodigy Game > Data Engineer, Game
0%: Toronto:  — Oakville, ON > Prodigy Game > Data Scientist, Game
0%: Toronto:  — Oakville, ON > Prodigy Game > Data Scientist, Marketing & Web Analytics
0%: Toronto:  — Toronto, ON > ThoughtStorm > Data Scientist/Analyst and Fraud

In [ ]:
from datetime import datetime

# sending it to csvs to save the data
now = datetime.now()
dt_string = now.strftime("%y%m%d%H%M")
filename = "Workopolis_" + dt_string + ".csv"
df.to_csv(filename , sep=',', encoding='utf-8')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cp filename /content/gdrive/My\ Drive

In [ ]:
from google.colab import files
files.download(filename)